In [1]:
import numpy as np
import matplotlib.pyplot as plt
from convergence_turbine import Francis_Turbine_test

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Druckrohrleitung.Druckrohrleitung_class_file import Druckrohrleitung_class

In [2]:
%matplotlib qt5

#Turbine
Q_nenn          = 0.85  # m³/s
p_nenn          = pressure_conversion(10.6,'bar','Pa')
closing_time    = 5    #s

#define constants pipe

g       = 9.81                  # gravitational acceleration [m/s²]
rho     = 1000.                 # density of water [kg/m³]

L       = 1000.                 # length of pipeline [m]
D       = 0.9                   # pipe diameter [m]
h_res   = 10.                   # water level in upstream reservoir [m]
n       = 50                  # number of pipe segments in discretization
nt      = 10000                    # number of time steps after initial conditions
f_D     = 0.01                  # Darcy friction factor
c       = 400.                  # propagation velocity of the pressure wave [m/s]
h_pipe  = 105.                  # hydraulic head without reservoir [m] 
alpha   = np.arcsin(h_pipe/L)   # Höhenwinkel der Druckrohrleitung 


# preparing the discretization and initial conditions
initial_flux    = 0.8                                    # m³/s
initial_level   = h_res                                   # m
dx              = L/n                                   # length of each pipe segment
dt              = dx/c                                  # timestep according to method of characterisitics
nn              = n+1                                   # number of nodes
pl_vec          = np.arange(0,nn,1)*dx                  # pl = pipe-length. position of the nodes on the pipeline
t_vec           = np.arange(0,nt,1)*dt                  # time vector
h_vec           = np.arange(0,nn,1)*h_pipe/n            # hydraulic head of pipeline at each node


# define constants reservoir
conversion_pressure_unit    = 'mWS'

area_base                   = 75.               # m²
area_pipe                   = (D/2)**2*np.pi    # m²
critical_level_low          = 0.                # m
critical_level_high         = 100.              # m

# make sure e-RK4 method of reservoir has a small enough timestep to avoid runaway numerical error
nt_eRK4                     = 1              # number of simulation steps of reservoir in between timesteps of pipeline              
simulation_timestep         = dt/nt_eRK4

In [3]:
V = Ausgleichsbecken_class(area_base,area_pipe,critical_level_low,critical_level_high,simulation_timestep)
V.set_steady_state(initial_flux,initial_level,conversion_pressure_unit)

pipe = Druckrohrleitung_class(L,D,n,alpha,f_D)
pipe.set_pressure_propagation_velocity(c)
pipe.set_number_of_timesteps(nt)
pipe.set_steady_state(initial_flux,initial_level,area_base,pl_vec,h_vec)


initial_pressure_turbine = pipe.get_current_pressure_distribution()[-1]
T1 = Francis_Turbine_test(Q_nenn,p_nenn,closing_time,timestep=dt)
T1.set_steady_state(initial_flux,initial_pressure_turbine)

In [4]:
# initialization for timeloop

level_vec       = np.zeros_like(t_vec)
level_vec[0]    = V.get_current_level()

# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = pipe.get_current_velocity_distribution()
p_old = pipe.get_current_pressure_distribution()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir are set manually and
        # through the time evolution of the reservoir respectively  
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.zeros_like(t_vec)
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)

# set the boundary conditions for the first timestep
v_boundary_res[0]   = v_old[0]
v_boundary_tur[0]   = v_old[-1] 
p_boundary_res[0]   = p_old[0]
p_boundary_tur[0]   = p_old[-1]


In [5]:
fig1,axs1 = plt.subplots(2,1)
axs1[0].set_title('Pressure distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ [mWS]')
lo_00, = axs1[0].plot(pl_vec,pressure_conversion(p_old,'Pa',conversion_pressure_unit),marker='.')
axs1[0].set_ylim([0.9*np.min(pressure_conversion(p_old,'Pa',conversion_pressure_unit)),1.1*np.max(pressure_conversion(p_old,'Pa',conversion_pressure_unit))])

axs1[1].set_title('Velocity distribution in pipeline')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$v$ [m/s]')
lo_01, = axs1[1].plot(pl_vec,v_old,marker='.')
# axs1[1].set_ylim([0.9*np.min(v_old),1.1*np.max(v_boundary_res)])

fig1.tight_layout()
plt.pause(1)

In [6]:

for it_pipe in range(1,nt):
# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial conditions for the reservoir time evolution calculted with e-RK4
    V.set_pressure(p_old[0])
    V.set_outflux(v_old[0]*area_pipe)
    # calculate the time evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(nt_eRK4):
        V.timestep_reservoir_evolution()                                                             
    level_vec[it_pipe] = V.get_current_level() 

    
    # set boundary conditions for the next timestep of the characteristic method
    p_boundary_res[it_pipe] = V.get_current_pressure()
    T1.set_pressure(p_old[-1])
    T1.converge(area_pipe,p_old[-2],v_old[-2],alpha,f_D,dt)
    v_boundary_tur[it_pipe] = T1.get_current_Q()/area_pipe

    # the the boundary conditions in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    pipe.v[0] = (pipe.v[0]+V.get_current_outflux()/area_pipe)/2
    p_boundary_tur[it_pipe] = pipe.get_current_pressure_distribution()[-1]
    v_boundary_res[it_pipe] = pipe.get_current_velocity_distribution()[0]

    # perform the next timestep via the characteristic method
    pipe.timestep_characteristic_method()

    # prepare for next loop
    p_old = pipe.get_current_pressure_distribution()
    v_old = pipe.get_current_velocity_distribution()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    lo_00.remove()
    lo_01.remove()
    # lo_02.remove()
        # plot new pressure and velocity distribution in the pipeline
    lo_00, = axs1[0].plot(pl_vec,pressure_conversion(p_old,'Pa', conversion_pressure_unit),marker='.',c='blue')
    lo_01, = axs1[1].plot(pl_vec,v_old,marker='.',c='blue')
    
    fig1.suptitle(str(round(t_vec[it_pipe],2)) + '/' + str(round(t_vec[-1],2)))
    fig1.canvas.draw()
    fig1.tight_layout()
    plt.pause(0.000001)

In [9]:
fig2,axs2 = plt.subplots(2,2)
axs2[0,0].set_title('Pressure Reservoir')
axs2[0,0].plot(t_vec,pressure_conversion(p_boundary_res,'Pa',conversion_pressure_unit))
axs2[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,0].set_ylabel(r'$p$ [mWS]')
axs2[0,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_res,'Pa',conversion_pressure_unit)),1.1*np.max(pressure_conversion(p_boundary_res,'Pa',conversion_pressure_unit))])

axs2[0,1].set_title('Velocity Reservoir')
axs2[0,1].plot(t_vec,v_boundary_res)
axs2[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[0,1].set_ylim([0.9*np.min(v_boundary_res),1.1*np.max(v_boundary_res)])

axs2[1,0].set_title('Pressure Turbine')
axs2[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,'Pa',conversion_pressure_unit))
axs2[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,0].set_ylabel(r'$p$ [mWS]')
axs2[1,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_tur,'Pa',conversion_pressure_unit)),1.1*np.max(pressure_conversion(p_boundary_tur,'Pa',conversion_pressure_unit))])

axs2[1,1].set_title('Velocity Turbine')
axs2[1,1].plot(t_vec,v_boundary_tur)
axs2[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[1,1].set_ylim([0.9*np.min(v_boundary_tur),1.1*np.max(v_boundary_tur)])

fig2.tight_layout()
plt.show()

In [8]:
plt.plot(level_vec)